In [1]:
import os
os.environ["HTTPS_PROXY"] = ""
os.environ["HTTP_PROXY"] = ""

In [2]:
import time
import pandas as pd
import numpy as np
import math

# internal tools
from dsgtools.reporting import make_format
from dsgtools.reporting import TableWriter
from dsgtools.reporting import freq
from dsgtools.reporting import bivariate

import warnings
warnings.filterwarnings('ignore')

In [41]:
path = "Analytics/Customer Projects/Elevate - proj 11167 (FP,FI Custom FRD)/File from the Customer/ELEVATE_FRAUD_TAGS.csv"
path_pii = "Analytics/Customer Projects/Elevate - proj 11167 (FP,FI Custom FRD)/File from the Customer/app_names.csv"

In [42]:
from dsgtools import azure
data = pd.read_adls(path, reader = pd.read_csv, dtype = str, encoding='iso-8859-1')
print(data.shape)
data_pii = pd.read_adls(path_pii, reader = pd.read_csv, dtype = str, encoding='iso-8859-1')
print(data_pii.shape)

(380110, 32)
(380110, 5)


In [43]:
data = data.merge(data_pii, on = "APP_GID")
print(data.shape)

(380110, 36)


In [44]:
data.APP_GID.is_unique

True

## Overall File

In [45]:
data[['APP_INPT_NM_FIRST', 'APP_INPT_NM_LAST', 'APP_INPT_HOME_ADDR_LN1', 'APP_INPT_HOME_ADDR_CITY', 'APP_INPT_HOME_ADDR_ST_CD', 
'APP_INPT_HOME_ADDR_ZIP_CD', 'APP_INPT_SSN_y', 'APP_INPT_DOB', 'APP_INPT_PHON_CELL_NUM', "APP_INPT_PHON_HOME_NUM"]].isnull().sum()

APP_INPT_NM_FIRST             0
APP_INPT_NM_LAST              1
APP_INPT_HOME_ADDR_LN1        0
APP_INPT_HOME_ADDR_CITY       0
APP_INPT_HOME_ADDR_ST_CD      0
APP_INPT_HOME_ADDR_ZIP_CD     0
APP_INPT_SSN_y               43
APP_INPT_DOB                  0
APP_INPT_PHON_CELL_NUM        0
APP_INPT_PHON_HOME_NUM        0
dtype: int64

In [46]:
Input_FirstName = "APP_INPT_NM_FIRST"
Input_LastName = "APP_INPT_NM_LAST"
Input_StreetAddress = "APP_INPT_HOME_ADDR_LN1"
Input_City = "APP_INPT_HOME_ADDR_CITY"
Input_State = "APP_INPT_HOME_ADDR_ST_CD"
Input_Zip = "APP_INPT_HOME_ADDR_ZIP_CD"
Input_SSN = "APP_INPT_SSN_y"

data["sufficient_input"] = np.where((data[Input_FirstName].notnull() & data[Input_LastName].notnull() & data[Input_StreetAddress].notnull() & data[Input_Zip].notnull()) | \
             (data[Input_FirstName].notnull() & data[Input_LastName].notnull() & data[Input_StreetAddress].notnull() & data[Input_City].notnull() & data[Input_State].notnull()) | \
             (data[Input_FirstName].notnull() & data[Input_LastName].notnull() & data[Input_SSN].notnull()), 1, 0)
freq(data["sufficient_input"])

,Count,Pct,Cuml Count,Cuml Pct
sufficient_input,,,,
0,1.0,0.000003,1.0,0.000003
1,380109.0,0.999997,380110.0,1.000000


In [47]:
data = data[data["sufficient_input"] == 1]
from datetime import datetime
data["APPDATETIME_fmt"] = pd.to_datetime(data.APP_VNT_DT.str.slice(0,9))
data["APPDATETIME_fmt_F"] = data['APPDATETIME_fmt'].astype(str).str.slice(0,4) + data['APPDATETIME_fmt'].astype(str).str.slice(5,7)
data["APPDATETIME_fmt_F_dt"] = data['APPDATETIME_fmt'].astype(str).str.slice(0,4) + data['APPDATETIME_fmt'].astype(str).str.slice(5,7) + data['APPDATETIME_fmt'].astype(str).str.slice(8,10)
freq(data["APPDATETIME_fmt_F"] )

,Count,Pct,Cuml Count,Cuml Pct
APPDATETIME_fmt_F,,,,
201901,7999.0,0.021044,7999.0,0.021044
201902,7715.0,0.020297,15714.0,0.041341
201903,6438.0,0.016937,22152.0,0.058278
201904,12642.0,0.033259,34794.0,0.091537
201905,14341.0,0.037729,49135.0,0.129266
201906,16249.0,0.042748,65384.0,0.172014
201907,22419.0,0.058980,87803.0,0.230994
201908,20495.0,0.053919,108298.0,0.284913
201909,10177.0,0.026774,118475.0,0.311687


In [48]:
data["APP_OFFR_LOAN_SVC_FPD_AGED_CNT_fmt"] = np.where(data.APP_OFFR_LOAN_SVC_FPD_AGED_CNT.isnull(), -1, np.where(data.APP_OFFR_LOAN_SVC_FPD_AGED_CNT == "1", 1, 0))
freq(data["APP_OFFR_LOAN_SVC_FPD_AGED_CNT_fmt"])

,Count,Pct,Cuml Count,Cuml Pct
APP_OFFR_LOAN_SVC_FPD_AGED_CNT_fmt,,,,
-1,37527.0,0.098727,37527.0,0.098727
0,315606.0,0.830304,353133.0,0.929031
1,26976.0,0.070969,380109.0,1.000000


In [49]:
data = data.sort_values(by = ['APP_INPT_NM_FIRST', 'APP_INPT_NM_LAST', 'APP_INPT_HOME_ADDR_LN1', 'APP_INPT_HOME_ADDR_CITY', 'APP_INPT_HOME_ADDR_ST_CD', 
'APP_INPT_HOME_ADDR_ZIP_CD', 'APP_INPT_SSN_y', 'APP_INPT_DOB', 'APP_INPT_PHON_CELL_NUM', 'APPDATETIME_fmt_F_dt', 'APP_OFFR_LOAN_SVC_FPD_AGED_CNT_fmt'], ascending = False)
print(data.shape)
data = data.drop_duplicates(subset = ['APP_INPT_NM_FIRST', 'APP_INPT_NM_LAST', 'APP_INPT_HOME_ADDR_LN1', 'APP_INPT_HOME_ADDR_CITY', 'APP_INPT_HOME_ADDR_ST_CD', 
'APP_INPT_HOME_ADDR_ZIP_CD', 'APP_INPT_SSN_y', 'APP_INPT_DOB', 'APP_INPT_PHON_CELL_NUM', 'APPDATETIME_fmt_F_dt',], keep = "first")
print(data.shape)

(380109, 41)
(379951, 41)


In [14]:
data.to_adls("Analytics/RnD Projects/Product RnD/Credit/202311-CRD-Model-Revalidations/Formatted_Files/Full_sample_formatted_w_flags/ready_to_be_formatted/elevate_11167_full_file.parquet",
             overwrite = True)

In [12]:
from dsgtools import azure
data = pd.read_adls("Analytics/RnD Projects/Product RnD/Credit/202311-CRD-Model-Revalidations/Formatted_Files/Full_sample_formatted_w_flags/ready_to_be_formatted/elevate_11167_full_file.parquet",)
print(data.shape)

(379951, 41)


In [13]:
from datetime import datetime
data["APP_INPT_DOB_fmt"] = pd.to_datetime(data.APP_INPT_DOB.str.slice(0,10), errors = 'coerce', format = "%m/%d/%Y")
data["APP_INPT_DOB_fmt_dt"] = data['APP_INPT_DOB_fmt'].astype(str).str.slice(0,4) + data['APP_INPT_DOB_fmt'].astype(str).str.slice(5,7) + data['APP_INPT_DOB_fmt'].astype(str).str.slice(8,10)

## Sample for validation

In [15]:
fmt = make_format(cuts = [-np.inf, 0, np.inf])
freq(data.LOAN_SVC_IND, data.APP_OFFR_AMT.astype(float), data.LOAN_FUND_AMT.astype(float), data.APP_OFFR_LOAN_SVC_FPD_AGED_CNT, 
     cross = False, observed = True, format = [None, fmt, fmt, None])

Count  \
LOAN_SVC_IND APP_OFFR_AMT LOAN_FUND_AMT APP_OFFR_LOAN_SVC_FPD_AGED_CNT             
1            1+           (-inf, 0.0]   0                                   18.0   
                                        Missing                              5.0   
                                        1                                    4.0   
                          (0.0, inf]    0                               306169.0   
                                        Missing                          36497.0   
                                        1                                26966.0   
             Missing      (0.0, inf]    0                                    5.0   
                                        Missing                              2.0   
0            1+           (-inf, 0.0]   0                                  450.0   
                                        Missing                             51.0   
             Missing      (-inf, 0.0]   0                                 8840.0   
                                        Missing                            944.0   

                                                                             Pct  \
LOAN_SVC_IND APP_OFFR_AMT LOAN_FUND_AMT APP_OFFR_LOAN_SVC_FPD_AGED_CNT             
1            1+           (-inf, 0.0]   0                               0.000047   
                                        Missing                         0.000013   
                                        1                               0.000011   
                          (0.0, inf]    0                               0.805812   
                                        Missing                         0.096057   
                                        1                               0.070972   
             Missing      (0.0, inf]    0                               0.000013   
                                        Missing                         0.000005   
0            1+           (-inf, 0.0]   0                               0.001184   
                                        Missing                         0.000134   
             Missing      (-inf, 0.0]   0                               0.023266   
                                        Missing                         0.002485   

                                                                        Cuml Count  \
LOAN_SVC_IND APP_OFFR_AMT LOAN_FUND_AMT APP_OFFR_LOAN_SVC_FPD_AGED_CNT               
1            1+           (-inf, 0.0]   0                                     18.0   
                                        Missing                               23.0   
                                        1                                     27.0   
                          (0.0, inf]    0                                 306196.0   
                                        Missing                           342693.0   
                                        1                                 369659.0   
             Missing      (0.0, inf]    0                                 369664.0   
                                        Missing                           369666.0   
0            1+           (-inf, 0.0]   0                                 370116.0   
                                        Missing                           370167.0   
             Missing      (-inf, 0.0]   0                                 379007.0   
                                        Missing                           379951.0   

                                                                        Cuml Pct  
LOAN_SVC_IND APP_OFFR_AMT LOAN_FUND_AMT APP_OFFR_LOAN_SVC_FPD_AGED_CNT            
1            1+           (-inf, 0.0]   0                               0.000047  
                                        Missing                         0.000061  
                                        1                               0.000071  
                          (0.0, inf]    0                               0.805883  
                  

In [16]:
data = data[(data.LOAN_SVC_IND =="1") & (data.APP_OFFR_AMT.astype(float) > 0) & (data.LOAN_FUND_AMT.astype(float) > 0) ]
print(data.shape)
freq(data.UPDATED_FLAG)

(369632, 43)


,Count,Pct,Cuml Count,Cuml Pct
UPDATED_FLAG,,,,
CHARGED OFF,107301.0,0.290291,107301.0,0.290291
FIRST PARTY FRAUD,22118.0,0.059838,129419.0,0.350129
THIRD PARTY FRAUD,1061.0,0.002870,130480.0,0.353000
Missing,239152.0,0.647000,369632.0,1.000000


In [17]:
data = data[~data.UPDATED_FLAG.isin(["FIRST PARTY FRAUD", "THIRD PARTY FRAUD"])]
print(data.shape)

(346453, 43)


In [18]:
freq(data.APP_OFFR_LOAN_SVC_FPD_AGED_CNT_fmt)

,Count,Pct,Cuml Count,Cuml Pct
APP_OFFR_LOAN_SVC_FPD_AGED_CNT_fmt,,,,
-1,36497.0,0.105345,36497.0,0.105345
0,294354.0,0.849622,330851.0,0.954966
1,15602.0,0.045034,346453.0,1.000000


In [19]:
data = data[data.APP_OFFR_LOAN_SVC_FPD_AGED_CNT_fmt!=-1]
print(data.shape)
data = data[data.APPDATETIME_fmt_F.astype(int)>=202101]
bivariate("APPDATETIME_fmt_F", "APP_OFFR_LOAN_SVC_FPD_AGED_CNT_fmt", df = data)

(309956, 43)


tag               APP_OFFR_LOAN_SVC_FPD_AGED_CNT_fmt                    \
stats                                              N      PctN     Sum   
APPDATETIME_fmt_F                                                        
202101                                        4359.0  0.033100   401.0   
202102                                        3249.0  0.024671   370.0   
202103                                        2614.0  0.019849   224.0   
202104                                        4373.0  0.033207   265.0   
202105                                       10048.0  0.076300   629.0   
202106                                       16329.0  0.123995   998.0   
202107                                       19019.0  0.144421  1336.0   
202108                                       20035.0  0.152136  1330.0   
202109                                       11892.0  0.090302   825.0   
202110                                       10345.0  0.078555   753.0   
202111                                       11099.0  0.084281   754.0   
202112                                        8360.0  0.063482   485.0   
202201                                        6050.0  0.045941   317.0   
202202                                        3919.0  0.029759   172.0   
Missing                                          0.0  0.000000     0.0   
Total                                       131691.0  1.000000  8859.0   

tag                                              
stats                  Mean       WoE        IV  
APPDATETIME_fmt_F                                
202101             0.091994  0.339799  0.004431  
202102             0.113881  0.577636  0.010585  
202103             0.085692  0.261929  0.001526  
202104             0.060599 -0.111630  0.000394  
202105             0.062600 -0.077022  0.000438  
202106             0.061118 -0.102548  0.001247  
202107             0.070246  0.046408  0.000317  
202108             0.066384 -0.014281  0.000031  
202109             0.069374  0.032992  0.000100  
202110             0.072789  0.084712  0.000585  
202111             0.067934  0.010465  0.000009  
202112             0.058014 -0.157968  0.001480  
202201             0.052397 -0.265761  0.002895  
202202             0.043889 -0.451885  0.005012  
Missing            0.000000  0.000000  0.000000  
Total              0.067271  0.000000  0.029050

In [20]:
data.to_adls("Analytics/RnD Projects/Product RnD/Credit/202311-CRD-Model-Revalidations/Formatted_Files/cleaned/Elevate_cleaned_11_21.parquet", overwrite = True)

In [21]:
data.to_adls("Analytics/RnD Projects/Product RnD/Credit/202311-CRD-Model-Revalidations/Formatted_Files/elevate_11_21.parquet", overwrite = True)